
# Feed Forward 
__1. All nodes are fully connected.__<br>
__2. Activation flows from input layer to output, without back loops.__ <Br>
__3. There is one layer between input.__<Br>
__4. [Towards Data Science Blog BY Andrew Tch for neural networks details ](https://towardsdatascience.com/the-mostly-complete-chart-of-neural-networks-explained-3fb6f2367464).__


### Titanic Dataset is used to implement Feed Forward
[Download Titanic Dataset](https://www.kaggle.com/c/titanic/data).


In [145]:
import numpy as np 
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers

In [186]:
def reading_test_train_data():
    train = pd.read_csv('./datasets/train.csv')
    test = pd.read_csv('./datasets/test.csv')
    return train,test


def adding_title_using_name(dataset):
    """
    Fetching Mr,Miss,Mrs title from name 
    """ 
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    return dataset


def converting_title_to_int(title):
    """
    Assigning Number to each title
    """
    title= title.strip().lower()
    if title == 'mr':
        return 1
    elif title =='miss':
        return 2
    elif title =='mrs':
        return 3
    else:
        return 0
    
def age_into_grp(dataset):
    """
    converting age into age_group as bins
    """
    bins = [0,11,18,22,27,33,40,66, 120]
    labels = [str(i) for i in range(1,len(bins))]
    print(labels)
    dataset['Age'] = pd.cut(dataset['Age'], bins, labels = labels,include_lowest = True)
    return dataset

In [187]:
train,test = reading_test_train_data()


In [188]:
# first 10 rows
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [189]:
# 891 rows 
# 12 columns /featurs including target
train.shape

(891, 12)

In [190]:
# only age count() is less then total df len means contains nan values
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


###  Manipulating --- AGE 

In [191]:
# filling age with age median
train['Age'] = train['Age'].fillna(train['Age'].median())

### ADDING NEW FEATURE 

In [192]:
# check who came with family or who came alone
train['with_family'] = train['SibSp'] + train['Parch']
train.loc[train['with_family'] > 0, 'not_alone'] = 0
train.loc[train['with_family'] == 0, 'not_alone'] = 1
train['not_alone'] = train['not_alone'].astype(int)
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,with_family,not_alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1


### Filling Embarked column

In [193]:
# checking Embarked data stats
train['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [194]:
train['Embarked'] = train['Embarked'].fillna('S')

In [195]:

train['Embarked'] = train['Embarked'].astype('category').cat.codes

### Manipulating FARE

In [196]:
train['Fare'].describe() 
train['Fare'] = train['Fare'].astype(int)

### Manipulating GENDER feature

In [197]:
train['Sex'].unique()

array(['male', 'female'], dtype=object)

In [198]:
train['Sex'] = train['Sex'].astype('category').cat.codes
train['Sex'].unique()

array([1, 0], dtype=int64)

### Manipulating Name and extracting Title from it
#### Then Converting them into category

In [199]:
train = adding_title_using_name(train)


In [200]:
train['Title'] = train['Title'].apply(converting_title_to_int)

In [201]:
train['Title'].unique()

array([1, 3, 2, 0], dtype=int64)

## converting age into bins - range

In [202]:
train.isna().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked       False
with_family    False
not_alone      False
Title          False
dtype: bool

In [203]:
train = age_into_grp(train)

['1', '2', '3', '4', '5', '6', '7', '8']


In [211]:
train['Age'].value_counts()

5    303
7    143
4    106
6    101
3     92
2     71
1     68
8      7
Name: Age, dtype: int64

In [212]:
train.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

KeyError: "['PassengerId' 'Name' 'Ticket' 'Cabin'] not found in axis"

In [213]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,with_family,not_alone,Title
0,0,3,1,3,1,0,7,2,1,0,1
1,1,1,0,6,1,0,71,0,1,0,3
2,1,3,0,4,0,0,7,2,0,1,2
3,1,1,0,6,1,0,53,2,1,0,3
4,0,3,1,6,0,0,8,2,0,1,1


## HANDLING TEST DATA AND PREPARING IT


In [214]:
# viewing test dataset
test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [215]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,29.599282,0.447368,0.392344,35.627188
std,120.810458,0.841838,12.703770,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


### Manipulting age for test dataset

In [216]:
# filling age with age median
test['Age'] = test['Age'].fillna(test['Age'].median())

### Creating With_familt and not_alone feature

In [217]:
train.isna().any()

Survived       False
Pclass         False
Sex            False
Age            False
SibSp          False
Parch          False
Fare           False
Embarked       False
with_family    False
not_alone      False
Title          False
dtype: bool

In [218]:
# check who came with family or who came alone
test['with_family'] = test['SibSp'] + test['Parch']
test.loc[test['with_family'] > 0, 'not_alone'] = 0
test.loc[test['with_family'] == 0, 'not_alone'] = 1
test['not_alone'] = test['not_alone'].astype(int)
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,with_family,not_alone
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,2,0


### Filling Embarked column in test dataset

In [219]:
# checking Embarked data stats
test['Embarked'].describe()

count     418
unique      3
top         S
freq      270
Name: Embarked, dtype: object

In [220]:
test['Embarked'] = test['Embarked'].fillna('S')
test['Embarked'] = test['Embarked'].astype('category').cat.codes

### Manipulating FARE  in test dataset

In [221]:
test['Fare'].describe() 
test['Fare'] = train['Fare'].astype(int)

### Manipulating gender feature in test dataset

In [222]:
test['Sex'] = test['Sex'].astype('category').cat.codes
test['Sex'].unique()

array([1, 0], dtype=int64)

### Manipulating Name and extracting Title from it in test dataset
#### Then Converting them into category

In [223]:
test = adding_title_using_name(test)
test['Title'] = test['Title'].apply(converting_title_to_int)
test['Title'].unique()

array([1, 3, 2, 0], dtype=int64)

### converting age into bins

In [225]:
test = age_into_grp(test)
test.head()

['1', '2', '3', '4', '5', '6', '7', '8']


TypeError: '<' not supported between instances of 'int' and 'str'

In [226]:
test.isna().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked       False
with_family    False
not_alone      False
Title          False
dtype: bool

In [227]:
test.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)

In [228]:
passengerId_test = test['PassengerId']

In [229]:
test.drop(['PassengerId'],axis=1,inplace=True)

In [230]:
train.dropna(inplace=True)
y_train = train['Survived']

In [231]:
train.drop(['Survived'],axis=1,inplace=True)

###  Create feed forward neural network to train data

In [232]:
def feed_foward_model(number_of_features):
    # Start neural network
    model = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    model.add(layers.Dense(units=16, activation='relu', input_shape=(number_of_features,)))

    # Add fully connected layer with a ReLU activation function
    model.add(layers.Dense(units=16, activation='relu'))

    # Add fully connected layer with a sigmoid activation function
    model.add(layers.Dense(units=1, activation='sigmoid'))
    
    return model

In [233]:
network = feed_foward_model(len(test.columns))
network.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])# Accuracy performance metric

In [234]:
#Fitting the data to the training dataset
network.fit(train,y_train, batch_size=10, epochs=300)

Epoch 1/300
891/891 [==============================] - 2s 2ms/step - loss: 1.7530 - acc: 0.5466
Epoch 2/300
891/891 [==============================] - 0s 162us/step - loss: 0.6213 - acc: 0.6801 0s - loss: 0.6164 - acc: 0.69
Epoch 3/300
891/891 [==============================] - 0s 141us/step - loss: 0.5940 - acc: 0.6914
Epoch 4/300
891/891 [==============================] - 0s 142us/step - loss: 0.5788 - acc: 0.6958
Epoch 5/300
891/891 [==============================] - 0s 172us/step - loss: 0.5558 - acc: 0.7194
Epoch 6/300
891/891 [==============================] - 0s 181us/step - loss: 0.5478 - acc: 0.7239
Epoch 7/300
891/891 [==============================] - 0s 278us/step - loss: 0.5282 - acc: 0.7430
Epoch 8/300
891/891 [==============================] - 0s 212us/step - loss: 0.5254 - acc: 0.7475
Epoch 9/300
891/891 [==============================] - 0s 195us/step - loss: 0.5184 - acc: 0.7587
Epoch 10/300
891/891 [==============================] - 0s 185us/step - loss: 0.5034 - acc

891/891 [==============================] - 0s 292us/step - loss: 0.4244 - acc: 0.8137
Epoch 82/300
891/891 [==============================] - 0s 282us/step - loss: 0.4139 - acc: 0.8204
Epoch 83/300
891/891 [==============================] - 0s 273us/step - loss: 0.4128 - acc: 0.8204
Epoch 84/300
891/891 [==============================] - 0s 297us/step - loss: 0.4164 - acc: 0.8204
Epoch 85/300
891/891 [==============================] - 0s 329us/step - loss: 0.4171 - acc: 0.8249
Epoch 86/300
891/891 [==============================] - 0s 268us/step - loss: 0.4215 - acc: 0.8137
Epoch 87/300
891/891 [==============================] - 0s 271us/step - loss: 0.4148 - acc: 0.8204
Epoch 88/300
891/891 [==============================] - 0s 283us/step - loss: 0.4104 - acc: 0.8316
Epoch 89/300
891/891 [==============================] - 0s 317us/step - loss: 0.4175 - acc: 0.8215
Epoch 90/300
891/891 [==============================] - 0s 260us/step - loss: 0.4078 - acc: 0.8294
Epoch 91/300
891/891 [=

891/891 [==============================] - 0s 201us/step - loss: 0.4047 - acc: 0.8249
Epoch 163/300
891/891 [==============================] - 0s 252us/step - loss: 0.4086 - acc: 0.8182
Epoch 164/300
891/891 [==============================] - 0s 198us/step - loss: 0.4125 - acc: 0.8193
Epoch 165/300
891/891 [==============================] - 0s 197us/step - loss: 0.4238 - acc: 0.8249
Epoch 166/300
891/891 [==============================] - 0s 188us/step - loss: 0.4072 - acc: 0.8215
Epoch 167/300
891/891 [==============================] - 0s 181us/step - loss: 0.4101 - acc: 0.8204
Epoch 168/300
891/891 [==============================] - 0s 178us/step - loss: 0.4050 - acc: 0.8204
Epoch 169/300
891/891 [==============================] - 0s 226us/step - loss: 0.4041 - acc: 0.8260
Epoch 170/300
891/891 [==============================] - 0s 188us/step - loss: 0.4073 - acc: 0.8227
Epoch 171/300
891/891 [==============================] - 0s 186us/step - loss: 0.4063 - acc: 0.8215
Epoch 172/300


891/891 [==============================] - 0s 267us/step - loss: 0.4039 - acc: 0.8272
Epoch 244/300
891/891 [==============================] - 0s 346us/step - loss: 0.4110 - acc: 0.8305
Epoch 245/300
891/891 [==============================] - 0s 202us/step - loss: 0.3843 - acc: 0.8395
Epoch 246/300
891/891 [==============================] - 0s 259us/step - loss: 0.3876 - acc: 0.8350
Epoch 247/300
891/891 [==============================] - 0s 190us/step - loss: 0.3922 - acc: 0.8384
Epoch 248/300
891/891 [==============================] - 0s 309us/step - loss: 0.3950 - acc: 0.8305
Epoch 249/300
891/891 [==============================] - 0s 199us/step - loss: 0.3856 - acc: 0.8305
Epoch 250/300
891/891 [==============================] - 0s 189us/step - loss: 0.3934 - acc: 0.8395
Epoch 251/300
891/891 [==============================] - 0s 313us/step - loss: 0.3945 - acc: 0.8305
Epoch 252/300
891/891 [==============================] - 0s 208us/step - loss: 0.4005 - acc: 0.8260
Epoch 253/300


In [235]:
model_evaluation=network.evaluate(train, y_train)
model_evaluation

891/891 [==============================] - 0s 216us/step


[0.3796709138569741, 0.8316498311815572]

In [237]:
y_pred=network.predict(test)
y_pred =(y_pred>0.5)

In [245]:
result = pd.DataFrame()

In [246]:
result['PassengerId'] = passengerId_test

In [247]:
result['Survived'] = y_pred

In [248]:
result['Survived'] = np.where(result['Survived'] == False,0,1)

In [252]:
result.to_csv("./datasets/results.csv",index=False)